# Modélisation d’un réseau BT – Trey
Par M. Buchser & M. Joye

Réalisé lors du semestre n°1 de 2024-2025



### Code of the work done

Import of librairies

In [11]:
import numpy as np
import pandas as pd
import pandapower as pp
import pp_heig_plot as pp_plot
import pp_heig_simulation as pp_sim
from datetime import time

Import the net from the Excel file

In [12]:
net_file_path = "data/trey_power_network.xlsx"
net = pp_sim.load_net_from_xlsx(file_path=net_file_path)
# net

Control that everything is okay

In [13]:
# net.bus
# net.res_line
# net.ext_grid

Plot the net

In [14]:
plot = pp_plot.plot_power_network(net, plot_title="Network of Trey")

Test power flow - plot with constant flow

In [15]:
pp.runpp(net)
pp_plot.plot_powerflow_result(net=net, plot_title="Network of Trey")

In [16]:
# net.res_bus

In [17]:
# net.res_line

In [18]:
# net.res_trafo

2.1.2 Identification and modeling of load curves

In [19]:
# Import datas from CSV file
consumer_file_path = "data/liste_des_batiments_vf.csv"
consumer_data = pd.read_csv(consumer_file_path, encoding="latin1")

# Regrouping consumers by cabinet
consumer_grouped_data = (
    consumer_data.groupby("Cabinet")
    .apply(lambda x: x.reset_index(drop=True))
    .reset_index(drop=True)
)

consumer_grouped_data

/tmp/ipykernel_14962/712015119.py:8: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Affectation,Empreinte au sol [m2],Cabinet,geometry,egid,street,npa,city,canton,building_status,building_cath,building_class,building_year,building_month,building_area,floor_nb,room_nb,pc_sheltter,sre,heat_1,heat_source_1,heat_info_1,heat_date_1,heat_2,heat_source_2,heat_info_2,heat_date_2,heatwater_1,heatwater_source_1,heatwater_info_1,heatwater_date_1,heatwater_2,heatwater_source_2,heatwater_info_2,heatwater_date_2,edid,entrance_nb,esid,entrance_east,entrance_north,ewid,housing_area,housing_room_nb,housing_status,housing_year,kitchen
0,bâtiment agricole,1177,60437,POINT (2560799 1179684),280063253,Chemin de Malafin 1.2,1552,Trey,VD,built,no_residential,farm,1995.0,NaN,1177,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,-,NaN,unheated,869,30.11.2020,NaN,unheated,869.0,30.11.2020,1,1.2,10041722,2560799.000,1179684.000,NaN,NaN,NaN,NaN,NaN,NaN
1,bâtiment agricole,657,CDBT003746,POINT (2560899 1180096),9031633,La Perrause 1.4,1552,Trey,VD,built,no_residential,farm,2002.0,NaN,657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,-,NaN,unheated,869,29.11.2001,NaN,unheated,869.0,29.11.2001,0,1.4,10068564,2560899.000,1180096.000,NaN,NaN,NaN,NaN,NaN,NaN
2,habitation,122,CDBT003746,POINT (2560978.6 1180136.8),825606,La Perrause 1,1552,Trey,VD,built,full_residential,one_housing,1989.0,NaN,122,2.0,NaN,NaN,NaN,boiler_1,mazout,855.0,16.04.2018,no_heater,unheated,855.0,16.04.2018,NaN,mazout,855,16.04.2018,NaN,unheated,855.0,16.04.2018,0,1,10068564,2560978.630,1180136.762,['1'],[244],[6],[3004],[1999],[1]
3,habitation,303,CDBT004760,POINT (2560773.9 1180111.1),825614,Impasse de la Forge 4,1552,Trey,VD,built,full_residential,multi_housing,NaN,NaN,303,2.0,NaN,NaN,NaN,boiler_1,mazout,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,Firewood,860,29.11.2001,NaN,NaN,NaN,-,1,4,10153657,2560773.900,1180111.116,['2'],[100],[4],[3004],[1999],[1]
4,habitation,223,CDBT004760,POINT (2560781.8 1180066.4),825608,Impasse de la Forge 1,1552,Trey,VD,built,full_residential,one_housing,NaN,NaN,223,2.0,NaN,NaN,NaN,boiler_1,mazout,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,mazout,860,29.11.2001,NaN,unheated,860.0,29.11.2001,0,1,10153657,2560781.804,1180066.380,['1'],[99],[6],[3004],[1999],[1]
5,habitation et rural,423,CDBT004760,POINT (2560827.7 1180093.5),825615,Route de Payerne 2,1552,Trey,VD,built,full_residential,one_housing,NaN,NaN,423,2.0,NaN,NaN,NaN,boiler_1,mazout,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,electricity,860,29.11.2001,NaN,unheated,860.0,29.11.2001,0,2,10146224,2560827.659,1180093.473,['1'],[100],[4],[3004],[1999],[1]
6,habitation,310,CDBT004764,POINT (2560838.8 1180044.6),3175197,Route de Payerne 4,1552,Trey,VD,built,full_residential,multi_housing,1618.0,NaN,310,2.0,NaN,NaN,NaN,boiler_1,mazout,864.0,31.10.2017,NaN,NaN,NaN,-,NaN,mazout,864,31.10.2017,NaN,unheated,860.0,29.11.2001,0,4,10146224,2560838.753,1180044.562,['1' '2' '3' '4' '5' '6'],[124 100 127 103 187 145],[3 3 3 3 4 4],[3004 3004 3004 3004 3004 3004],[1999 1999 1999 1999 1999 1999],[1 1 1 1 1 1]
7,bâtiment ecclésiastique,128,CDBT004764,POINT (2560838 1180024),280010796,Place du Collège 1.2,1552,Trey,VD,built,no_residential,church,1800.0,NaN,128,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,-,NaN,unheated,869,30.11.2020,NaN,unheated,869.0,30.11.2020,1,1.2,10141605,2560838.000,1180024.000,NaN,NaN,NaN,NaN,NaN,NaN
8,habitation,79,CDBT004764,POINT (2560875.8 1179986),825603,Route de Payerne 7,1552,Trey,VD,built,full_residential,one_housing,NaN,NaN,79,2.0,NaN,NaN,NaN,boiler_1,Firewood,860.0,29.11.2001,NaN,NaN,NaN,-,NaN,Firewood,869,18.07.2013,NaN,unheated,860.0,29.11.2001,0,7,10146224,2560875.838,1179985.975,['1'],[100],[3],[3004],[1999],[1]
9,"dépendance, bâtiment",90,CDBT004764,POINT (2560861 1179995),280014120,Route de Payerne 3.1,1552,Trey,VD,built,no_residential,industial,NaN,NaN,90,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,-,NaN,unheated,869,30.11.2020,NaN,unheated,869.0,30.11.2020,1,3.1,10146224,2560861.000,1179995.000,NaN,NaN,NaN,NaN,NaN,NaN


Import the power profile from the Excel file

In [20]:
# power_profile_file_path = "data/power_profile.xlsx"
# power_profile = pp_sim.load_power_profile_form_xlsx(file_path=power_profile_file_path)

Apply the profile and the net

In [21]:
# apply = pp_sim.apply_power_profile(power_profile)

Simulation

In [22]:
# pp_sim.run_time_simulation(net)